In [2]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import AutoTokenizer, AutoModel
from utils.MIND import MIND, MIND_news
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [3]:
t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")
# t3 = AutoTokenizer.from_pretrained('google/bigbird-roberta-base', cache_dir=config.path + "bert_cache/")

In [6]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'
# config.reducer = 'first'

# config.signal_length = 10
config.scale = 'large'
config.impr_size = 1000
config.mode = 'inspect'
config.news = 28
# config.fast = True
# config.news = 21563

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
# file_directory = manager.path + "MIND/MIND{}_{}/".format(manager.scale, manager.mode)
file_directory = manager.path + "MIND/MIND{}_dev/".format(manager.scale)
# manager.gather_same_user_impr()

a = MIND(manager, file_directory)
# b = MIND_news(manager)

[2021-11-04 10:08:45,610] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDlarge_dev/1000/behaviors.pkl
[2021-11-04 10:08:50,194] INFO (utils.MIND) rank 0 extracting users who browsed news 28
[2021-11-04 10:08:50,302] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDlarge_dev/news.pkl
[2021-11-04 10:08:51,018] INFO (utils.utils) deduplicating...


In [13]:
a[0]

{'impr_index': 224411,
 'user_id': 153998,
 'cdd_id': array([67843, 68087, 65140, 67483, 69773, 67720, 68076,   250, 63996,
        68755, 65265, 64813, 67156, 61849]),
 'his_id': array([71567, 63227, 70422, 43270, 31519, 54133, 49069, 27154, 32061,
        59445, 41719,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]),
 'cdd_encoded_index': array([[  101,  2129,  2000, ...,     0,     0,     0],
        [  101, 12084, 10584, ...,     0,     0,     0],
        [  101,  2152, 22487, ...,     0,     0,     0],
        ...,
        [  101,  7163, 12525, ...,     0,     0,     0],
        [  101, 12669,  4819, ...,     0,     0,     0],
        [  101,  2382,  2190, ...,     0,     0,     0]]),
 'his_encoded_index': array([[  101,  4586,

In [ ]:
# manager.construct_whole_dataset()

In [2]:
# behaviors = pickle.load(open('data/cache/bert/MINDlarge_dev/1000/behaviors.pkl','rb'))
# news = pickle.load(open('data/cache/bert/MINDdemo_train/news.pkl','rb'))
news = pickle.load(open('data/cache/bert/MINDlarge_train/news.pkl','rb'))

enc = news['encoded_news']

In [1]:
len("N77613 N5324 N88129 N26834 N121794 N85484 N16078 N85722 N74147 N99932 N67397 N121551 N82799 N67770 N9375 N65119 N9375 N124922 N70796 N127337 N85056 N127659 N126802 N62383 N88290 N115743 N122766 N60882 N29907 N8427 N25353 N45946 N122920 N26947 N107290 N77265 N108319".split())

37

In [6]:
news = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/bert/MINDdemo_train/news.pkl', 'rb'))
ids = news['encoded_news']
subwords = news['subwords_all']

index = 37405
news = t.convert_ids_to_tokens(a.encoded_news_original[index])
mask = a.attn_mask_original[index]
subword = a.subwords_original[index]

for i,j,k in zip(news,mask,subword):
    print(i,j,k)

AttributeError: 'MIND' object has no attribute 'encoded_news_original'

In [7]:
# bm25, entity, first
index = 1

# news = t.convert_ids_to_tokens(a[index]['his_encoded_index'][1])
# mask = a[index]['his_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
# subword = a[index]['his_subword_index'][1]

# for i,j,k in zip(news,mask,subword):
#     print(i,j,k)
res = a[index]
ids = res['cdd_id'][1]
news = t.convert_ids_to_tokens(res['cdd_encoded_index'][1])
mask = res['cdd_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
# subword = res['cdd_subword_index'][1]
print(ids)
for i,j,k in zip(news,mask):
    print(i,j,k)

KeyError: 'cdd_subword_index'

In [9]:
news = t.convert_ids_to_tokens(a[0]['his_encoded_index'][1])
mask = a[0]['his_attn_mask'][1]
word = manager.convert_tokens_to_words(news)
# subword = a[0]['his_subword_index'][1]
dedup = a[0]['his_refined_mask'][1]

for i,j,k in zip(news,mask,dedup):
    print(i,j,k)

[CLS] 1 1
three 1 1
take 1 1
##away 1 1
##s 1 1
from 1 1
yankees 1 1
' 1 1
al 1 1
##cs 1 1
game 1 1
5 1 1
victory 1 1
over 1 1
the 1 1
astros 1 1
the 1 0
yankees 1 0
kept 1 1
hope 1 1
alive 1 1
thanks 1 1
to 1 1
some 1 1
impressive 1 1
starting 1 1
pitching 1 1
and 1 1
a 1 1
pair 1 1
of 1 1
early 1 1
home 1 1
runs 1 1
. 1 0
baseball 1 1
_ 1 0
mlb 1 1
[SEP] 1 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
[PAD] 0 0
